In [1]:
# Import Required Libraries
import json
import pandas as pd
import os
import subprocess
from DatabasePackage import assign_asset_to_player, excel_to_assets_json

In [2]:
# Generate Asset Database from Excel File
# This cell reads the Asset_database.xlsx file and converts it to a JSON format
# with the structure: {Area: {Asset: {land_price, house_price, rent {...}}}}
# The resulting Asset_database.json file is created in the current directory
excel_to_assets_json("Asset_database.xlsx")

# Comprehensive Test Cases for Asset Assignment System

## Overview
This test suite validates the core functionality of the Monopoly player asset assignment system. It tests the system's ability to manage player properties, enforce business rules, and maintain data integrity across multiple scenarios.

## Test Execution Flow

The test cases run sequentially using dynamically loaded data from the Asset Database to ensure compatibility with any asset configuration:

1. **Initialization**: Cleans up previous test data and loads available areas and assets
2. **Sequential Assignment**: Assigns assets to players in a controlled manner
3. **Conflict Testing**: Attempts invalid operations to verify error handling
4. **Verification**: Checks the final state for data consistency

## Test Case Details

### Step 1: Player 1 Asset Assignment
- **Objective**: Establish initial property ownership for Player 1
- **Significance**: Validates basic asset assignment functionality and database initialization
- **Expected Result**: First two assets from the first area are assigned to Player 1 with 4 and 3 houses respectively

### Step 2: Player 2 Asset Assignment  
- **Objective**: Assign different properties to a second player
- **Significance**: Tests multi-player support and ensures assets can be distributed across different players
- **Expected Result**: Assets are assigned to Player 2 (from different area if available, or different assets from same area)

### Step 3: Conflict Detection - Duplicate Asset Assignment
- **Objective**: Attempt to assign an asset already owned by Player 1 to Player 2
- **Significance**: Validates conflict detection mechanism that prevents duplicate ownership
- **Expected Result**: ERROR message displayed, assignment denied, Player 1 retains ownership

### Step 4: House Cap Enforcement
- **Objective**: Try to assign more than 4 houses to an asset
- **Significance**: Validates the business rule that properties cannot have more than 4 houses
- **Expected Result**: WARNING displayed, total houses capped at 4 maximum

### Step 5: House Accumulation
- **Objective**: Add additional houses to an asset already owned by Player 1
- **Significance**: Tests the additive nature of house assignments (adding to existing rather than replacing)
- **Expected Result**: Houses are added to existing count with appropriate info messages

### Step 6: Invalid Asset Validation
- **Objective**: Attempt to assign a non-existent asset to a player
- **Significance**: Validates asset existence checking against the Asset Database
- **Expected Result**: ERROR message displayed, assignment denied due to invalid asset

## Key Features Tested

| Feature | Test Case | Validation |
|---------|-----------|-----------|
| Basic Assignment | Steps 1-2 | Assets can be assigned successfully |
| Duplicate Prevention | Step 3 | System prevents duplicate ownership |
| House Cap | Step 4 | Maximum of 4 houses enforced |
| House Accumulation | Step 5 | Additive assignment works correctly |
| Asset Validation | Step 6 | Only valid assets accepted |
| Data Integrity | Final Verification | No duplicates exist in database |

## Expected Output

The test produces detailed console output showing:
- Successful assignments (SUCCESS messages)
- Failed operations with clear error messages (ERROR messages)
- Warnings for boundary violations (WARNING messages)
- Status updates (INFO messages)
- Final database state in formatted JSON
- Verification result confirming no duplicate assignments

In [4]:
# Test Case: Using Command-Line Interface
# This test uses the player_asset_assignment.py script with command-line arguments
# instead of calling the function directly

print("=== Test Case: Asset Conflict Detection with Area-Asset Hierarchy ===\n")

# Define test database file paths
test_db_file = "Player_database.json"
asset_db_file = "Asset_database.json"

# Remove test file if it exists from previous test run
if os.path.exists(test_db_file):
    os.remove(test_db_file)
    print(f"Removed old test database file: {test_db_file}\n")

# First, generate the asset database from Excel if it doesn't exist
if not os.path.exists(asset_db_file):
    print(f"Generating asset database from Excel...\n")
    excel_to_assets_json("Asset_database.xlsx", asset_db_file)

# Load and display available areas and assets
with open(asset_db_file, 'r', encoding='utf-8') as f:
    asset_db = json.load(f)

print("Available Areas and Assets in Asset Database:")
print("-" * 50)
for area, assets in asset_db.items():
    print(f"Area: {area}")
    for asset_name in assets.keys():
        print(f"  - {asset_name}")
print()

print("Step 1: Assigning assets to Player 1")
print("-" * 50)
area1 = list(asset_db.keys())[0]
asset1_list = list(asset_db[area1].keys())
subprocess.run(["python", "player_asset_assignment.py", "--player", "Player 1", "--area", area1, "--asset", asset1_list[0], "--houses", "4", "--player_db", test_db_file, "--asset_db", asset_db_file])
subprocess.run(["python", "player_asset_assignment.py", "--player", "Player 1", "--area", area1, "--asset", asset1_list[1], "--houses", "3", "--player_db", test_db_file, "--asset_db", asset_db_file])

print("\nStep 2: Assigning assets to Player 2")
print("-" * 50)
if len(asset_db) > 1:
    area2 = list(asset_db.keys())[1]
    asset2_list = list(asset_db[area2].keys())
    subprocess.run(["python", "player_asset_assignment.py", "--player", "Player 2", "--area", area2, "--asset", asset2_list[0], "--houses", "1", "--player_db", test_db_file, "--asset_db", asset_db_file])
    subprocess.run(["python", "player_asset_assignment.py", "--player", "Player 2", "--area", area2, "--asset", asset2_list[1], "--houses", "0", "--player_db", test_db_file, "--asset_db", asset_db_file])
else:
    subprocess.run(["python", "player_asset_assignment.py", "--player", "Player 2", "--area", area1, "--asset", asset1_list[2], "--houses", "1", "--player_db", test_db_file, "--asset_db", asset_db_file])
    subprocess.run(["python", "player_asset_assignment.py", "--player", "Player 2", "--area", area1, "--asset", asset1_list[3], "--houses", "0", "--player_db", test_db_file, "--asset_db", asset_db_file])

# Load and display current database state
with open(test_db_file, 'r', encoding='utf-8') as f:
    test_db = json.load(f)

print("\nCurrent database state:")
print(json.dumps(test_db, indent=2))

print("\n" + "="*50)
print("Step 3: Testing Conflict - Try to assign asset from Player 1 to Player 2")
print("-" * 50)
subprocess.run(["python", "player_asset_assignment.py", "--player", "Player 2", "--area", area1, "--asset", asset1_list[0], "--houses", "2", "--player_db", test_db_file, "--asset_db", asset_db_file])

print("\n" + "="*50)
print("Step 4: Testing House Cap - Try to assign more than 4 houses")
print("-" * 50)
subprocess.run(["python", "player_asset_assignment.py", "--player", "Player 1", "--area", area1, "--asset", asset1_list[0], "--houses", "5", "--player_db", test_db_file, "--asset_db", asset_db_file])

print("\n" + "="*50)
print("Step 5: Testing House Addition - Add houses to existing asset (Player 1)")
print("-" * 50)
subprocess.run(["python", "player_asset_assignment.py", "--player", "Player 1", "--area", area1, "--asset", asset1_list[0], "--houses", "2", "--player_db", test_db_file, "--asset_db", asset_db_file])

print("\n" + "="*50)
print("Step 6: Testing Invalid Asset - Try to assign non-existent asset")
print("-" * 50)
subprocess.run(["python", "player_asset_assignment.py", "--player", "Player 1", "--area", area1, "--asset", "Non-existent Asset", "--houses", "2", "--player_db", test_db_file, "--asset_db", asset_db_file])

# Load and display final database state
with open(test_db_file, 'r', encoding='utf-8') as f:
    test_db = json.load(f)

print("\n" + "="*50)
print("Final database state (with accumulated houses):")
print(json.dumps(test_db, indent=2))

# Verify no duplicates exist
print("\n" + "="*50)
print("Verification: Checking for duplicate assignments")
print("-" * 50)
all_assets = set()
has_duplicates = False
for player, areas in test_db.items():
    for area, assets in areas.items():
        for asset in assets:
            asset_key = f"{area}::{asset}"
            if asset_key in all_assets:
                print(f"DUPLICATE FOUND: {asset} in area {area} is assigned to multiple players!")
                has_duplicates = True
            else:
                all_assets.add(asset_key)

if not has_duplicates:
    print("✓ No duplicate assignments detected. All assets are uniquely assigned.")
else:
    print("✗ Duplicate assignments detected!")

=== Test Case: Asset Conflict Detection with Area-Asset Hierarchy ===

Removed old test database file: Player_database.json

Available Areas and Assets in Asset Database:
--------------------------------------------------
Area: Street 1
  - Asset 1.1
  - Asset 1.2
  - Asset 1.3
  - Asset 1.4
Area: Street 2
  - Asset 2.1
  - Asset 2.2
  - Asset 2.3
  - Asset 2.4

Step 1: Assigning assets to Player 1
--------------------------------------------------
INFO: Created new player database (file 'Player_database.json' did not exist)
SUCCESS: Asset 'Asset 1.1' (Street 1) assigned to player 'Player 1' with 4 houses
INFO: Player database saved to 'Player_database.json'
INFO: Loaded player database from 'Player_database.json'
SUCCESS: Asset 'Asset 1.2' (Street 1) assigned to player 'Player 1' with 3 houses
INFO: Player database saved to 'Player_database.json'

Step 2: Assigning assets to Player 2
--------------------------------------------------
INFO: Loaded player database from 'Player_database.